In [ ]:
#Dataset download code
import os
import gdown
import tarfile

# Define the URL for the UTKFace dataset
url = 'https://drive.google.com/u/0/uc?id=0BxYys69jI14kYVM3aVhKS1VhRUk&export=download'

# Define the directory to store the dataset
data_dir = './data'

# Create the directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the tar file
filename = 'UTKFace.tar.gz'
filepath = os.path.join(data_dir, filename)
gdown.download(url, filepath, quiet=False)

# Extract the tar file
tar = tarfile.open(filepath, 'r:gz')
tar.extractall(data_dir)
tar.close()


In [ ]:
#model generation
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# Define the path to the UTKFace dataset
data_dir = './data'

# Define the parameters for loading the images
img_height = 200
img_width = 200
batch_size = 32

# Define the training and validation datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# Define the model architecture
model = Sequential([
  Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

# Save the model
model.save('age_gender_prediction_model.h5')

In [ ]:
#app.py
import streamlit as st
import cv2
import tensorflow as tf
import numpy as np
# Load the trained model
model = tf.keras.models.load_model('age_gender_prediction_model.h5')

# Define the labels for gender prediction
gender_labels = ['Male', 'Female']

# Define the function to make predictions
def predict_age_and_gender(image):
    # Resize the image
    resized_image = cv2.resize(image, (200, 200))
    # Convert to float32
    image = np.float32(resized_image)
    # Normalize the image
    image /= 255.0
    # Add batch dimension
    image = np.expand_dims(image, axis=0)
    # Make the prediction
    prediction = model.predict(image)
    # Get the predicted age
    age = int(prediction[0])
    # Get the predicted gender
    gender_prediction = model.predict_classes(image)
    gender = gender_labels[gender_prediction[0][0]]
    # Return the predicted age and gender
    return age, gender

# Define the Streamlit app
def app():
    # Set the app title
    st.set_page_config(page_title='Age and Gender Prediction', page_icon='👴👩')
    st.title('Age and Gender Prediction')
    
    # Allow the user to upload an image
    uploaded_file = st.file_uploader('Choose an image', type=['jpg', 'jpeg', 'png'])
    if uploaded_file is not None:
        # Load the image
        image = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        # Make the prediction
        age, gender = predict_age_and_gender(image)
        # Display the image and prediction
        st.image(image, caption=f'Predicted Age: {age}, Predicted Gender: {gender}', use_column_width=True)

# Run the Streamlit app
if __name__ == '__main__':
    app()